This notebook takes the data set from the Starbucks customer datasets and produces different tables ready for cleaning and modeling.

- **Author**: Martín Pons
- **Date**: 2022-03-11

In [1]:
import numpy as np
import pandas as pd
import warnings
import time

from StarbucksAnalysis.Customer import Customer

warnings.filterwarnings('ignore')

# Load data

- `portfolio` contains de different offer types and their features
- `profile` contains customer features
- `transcript` containts information regarding about every offer and transaction event for each customer

In [2]:
portfolio = pd.read_json('raw_data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('raw_data/profile.json', orient='records', lines=True)
transcript = pd.read_json('raw_data/transcript.json', orient='records', lines=True)

In [3]:
profile.gender.value_counts(normalize = True)

M    0.572277
F    0.413423
O    0.014300
Name: gender, dtype: float64

In [4]:
transcript.shape

(306534, 4)

In [5]:
len(transcript.person.unique())

17000

In [6]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [7]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [8]:
display(transcript.sort_values(["person", "time"]).head(10))

,person,event,value,time
55972,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},168
77705,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},192
89291,0009655768c64bdeb2e877511632db8f,transaction,{'amount': 22.16},228
113605,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},336
139992,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '3f207df678b143eea3cee63160fa8bed'},372
153401,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},408
168412,0009655768c64bdeb2e877511632db8f,transaction,{'amount': 8.57},414
168413,0009655768c64bdeb2e877511632db8f,offer completed,{'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d...,414
187554,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},456
204340,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},504


# Cleaning `transcript` data frame

## unify `offer_id` key

In the value column some 'offer id' have an underscore, we'll substitute it for a blank space

In [9]:
def replace_offer_key(offer):
    
    if "offer_id" in offer.keys():
        offer["offer id"] = offer["offer_id"]
        del offer["offer_id"]
        
    return offer

transcript['value'] = transcript.value.apply(lambda x: replace_offer_key(x))

## Merge 'transcript' and 'portfolio'

It is useful for reasons explained later, to join `portfolio` and `transcripts` DataFrames

In [10]:
transcript['offer_id'] = transcript.apply(lambda x: x.value['offer id'] if "offer" in x.event else "Not an offer event", axis = 1)

transcript = transcript.merge(portfolio, how = "left", left_on = "offer_id", right_on = "id")

# drop original id offer column
transcript = transcript.drop("id", axis = 1)


## Change duration units

The offer duration is in days. We'll proceed to change it to hours so it's in the same units than the column `time`

In [11]:
transcript["duration"] = transcript["duration"] * 24

## Change person id to integer type

`id`as it is in the original `profile` data frame makes inneffient when filtering each customer. In te process of rearranging the `transcript` data frame we'll have to go through each customer to make a series of computations. We have 17.000 constumers, the operation consumes time, specially using that long  `id` string. If we switch to an integer key, the process lasts considerably less time

In [12]:
# setting the id value for the first customer
new_id = 1

# getting customers id
customers = np.unique(transcript.person)

# initiate a dictionary of old keys with none values
customers_id_dict =  dict.fromkeys(customers, None)

# initiate empty list to fill with the new customer id
customers_id = list()

# assign the new corrlative customer id in the transcripts dataframe if the id hasn't previously detected before
for idx in transcript.person:
    if customers_id_dict[idx] is None:
        customers_id_dict[idx] = new_id
        new_id += 1
    customers_id.append(customers_id_dict[idx])
        
transcript["person"] = customers_id     
profile["new_id"] = profile.id.apply(lambda x: customers_id_dict[x])

In [13]:
transcript.head()

,person,event,value,time,offer_id,reward,channels,difficulty,duration,offer_type
0,1,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0,"[web, email, mobile]",5.0,168.0,bogo
1,2,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0b1e1539f2cc45b7b9fa7c272da2e1d7,5.0,"[web, email]",20.0,240.0,discount
2,3,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,2906b810c7d4411798c6938adc9daaa5,2.0,"[web, email, mobile]",10.0,168.0,discount
3,4,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,fafdcd668e3743c1bb461111dcafc2a4,2.0,"[web, email, mobile, social]",10.0,240.0,discount
4,5,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,4d5c57ea9a6940dd891ad53e9dbe8da0,10.0,"[web, email, mobile, social]",10.0,120.0,bogo


# Agregate and rearrange data

Getting the completion status for each customer and adding some additional features

In [14]:
customers_list = np.unique(transcript["person"])

For every customer wt get the completion status using the class Customer, which is a data frame, and stack one over the other

In [15]:
import time
start = time.time()
previous_time = time.time()


n_customer = 0

# initiate offers dataframe
offers_df = pd.DataFrame()

# getting completion status with Customer class for each client
for customer in customers_list:
    if n_customer % 1000 == 0: 
        
        print("Customer nº: ", n_customer)
        current_time = time.time()
        print("Elapsed time: ", current_time - previous_time)
        previous_time = current_time
        
    current_client = Customer(transcript, customer) # creation of Customer instance
    offers_completed = current_client.get_completion_status() # completion status dataframe
    offers_df = pd.concat([offers_df, offers_completed]) # concat current data frame with the main one
    n_customer += 1
    
end = time.time()
print("Elapsed time: ", end - start)

Customer nº:  0
Elapsed time:  0.0008561611175537109
Customer nº:  1000
Elapsed time:  9.278148651123047
Customer nº:  2000
Elapsed time:  9.528460025787354
Customer nº:  3000
Elapsed time:  9.821799516677856
Customer nº:  4000
Elapsed time:  9.819597959518433
Customer nº:  5000
Elapsed time:  9.777544260025024
Customer nº:  6000
Elapsed time:  9.81319522857666
Customer nº:  7000
Elapsed time:  10.020500898361206
Customer nº:  8000
Elapsed time:  10.013746500015259
Customer nº:  9000
Elapsed time:  10.687060832977295
Customer nº:  10000
Elapsed time:  11.167527198791504
Customer nº:  11000
Elapsed time:  10.814788818359375
Customer nº:  12000
Elapsed time:  10.893054723739624
Customer nº:  13000
Elapsed time:  10.92290449142456
Customer nº:  14000
Elapsed time:  11.188390016555786
Customer nº:  15000
Elapsed time:  11.5223069190979
Customer nº:  16000
Elapsed time:  11.633053302764893
Elapsed time:  178.46526336669922


In [18]:
offers_df = offers_df.merge(profile.drop("id", axis = 1), how = "left", left_on = "client_id", right_on = "new_id")
offers_df = offers_df.merge(portfolio, how = "left", left_on = "offer_id", right_on = "id").drop("id", axis = 1)

In [19]:
offers_df.to_csv("processed_data/offers_df.csv")